In [1]:
from environment import Environment
from magiccube import Cube
from agents import A2C, DQN, PPO
import yaml
import numpy as np
import itertools
import torch
import torch.nn as nn

In [2]:
args = yaml.safe_load(open("config.yaml", "r"))

env = Environment(
    method="LBL",
    size=3,
    args=args["environment"]
)

agent = DQN(env, args["DQN"])

In [3]:
cube = Cube()
print(cube)

          W  W  W                   
          W  W  W                   
          W  W  W                   
 O  O  O  G  G  G  R  R  R  B  B  B 
 O  O  O  G  G  G  R  R  R  B  B  B 
 O  O  O  G  G  G  R  R  R  B  B  B 
          Y  Y  Y                   
          Y  Y  Y                   
          Y  Y  Y                   



In [4]:
env.algorithm.status()

array([[1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1]])

In [72]:
#env.scramble()
env.step("B")
print(env.state2)
#print(env.state2.shape)
#print(env.algorithm.status())
print(env.cube)
print(agent.policy_net(agent.state_to_tensor(env.state2)))

[[[ 3  5  0  0 19]
  [ 1  3 -1  1  1]
  [ 4  1  2  2  6]
  [ 1 -1  4  3  3]
  [ 1 -1 -1  4  4]
  [ 4 -1  2  5 14]
  [ 3  4  0  6 17]
  [ 0  4 -1  7 20]
  [ 0  5  2  8 25]
  [-1  1  5  9  5]
  [-1  3 -1 10 10]
  [-1  3  5 11 11]
  [-1 -1  4 12 12]
  [-1 -1  5 13 13]
  [-1  3  4 14  9]
  [-1  2 -1 15 15]
  [-1  2  1 16  7]
  [ 4  3  1 17  0]
  [ 0  3 -1 18 18]
  [ 1  5  3 19  2]
  [ 2 -1  5 20 16]
  [ 0 -1 -1 21 21]
  [ 5 -1  0 22 22]
  [ 2  5  1 23  8]
  [ 2  0 -1 24 24]
  [ 4  2  0 25 23]]]
          B  Y  G                   
          B  W  R                   
          G  W  W                   
 Y  R  R  W  O  R  B  W  W  O  B  R 
 O  O  B  W  G  R  G  R  W  G  B  B 
 Y  O  B  W  G  Y  O  R  B  O  G  R 
          O  Y  G                   
          Y  Y  Y                   
          G  O  Y                   

tensor([1, 1, 0, 1])
tensor([[ 0.0691, -0.0760,  0.0080, -0.1121, -0.0430, -0.0202,  0.0529,  0.0835,
         -0.0480, -0.0159,  0.1221,  0.0880]], grad_fn=<AddmmBackwar

In [7]:
hidden_size = 128

embedding = nn.Linear(5, hidden_size)

attention = nn.MultiheadAttention(
    embed_dim=hidden_size,
    num_heads=4,
    batch_first=True
)

layer_norm = nn.LayerNorm(hidden_size)

ffn = nn.Sequential(
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(inplace=True)
)

output_layer = nn.Sequential(
    nn.Linear(hidden_size * 26, hidden_size * 12),
    nn.ReLU(inplace=True),
    nn.Linear(hidden_size * 12, hidden_size * 6),
    nn.ReLU(inplace=True),
    nn.Linear(hidden_size * 6, hidden_size * 3),
    nn.ReLU(inplace=True),
    nn.Linear(hidden_size * 3, hidden_size),
    nn.ReLU(inplace=True),
    nn.Linear(hidden_size, hidden_size // 2),
    nn.ReLU(inplace=True),
    nn.Linear(hidden_size // 2, 12)
)


In [8]:
state_tensor = torch.from_numpy(env.state2).float()
embedded = embedding(state_tensor)
attn_output, _ = attention(embedded, embedded, embedded)
attn_output = layer_norm(embedded + attn_output)
ffn_output = ffn(attn_output)
output = layer_norm(attn_output + ffn_output)

flattened = output.view(1, -1)
output = output_layer(flattened)

output

tensor([[-0.0348,  0.0706, -0.0873, -0.0922,  0.0188,  0.0744,  0.1190, -0.0996,
          0.0783, -0.0687,  0.1249, -0.0927]], grad_fn=<AddmmBackward0>)

In [9]:
# Definiamo un layer lineare: input con 3 feature, output con 2 feature
fc1 = nn.Linear(in_features=3, out_features=256)
fc2 = nn.Linear(in_features=256, out_features=2)

# Esempio di input: batch di 4 esempi, ciascuno con 3 feature
input_matrix = torch.tensor([
    [1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0],
    [7.0, 8.0, 9.0],
    [10.0, 11.0, 12.0]
])  # shape: (4, 3)

# Passaggio della matrice al layer
output = fc1(input_matrix)
print(output)
output = fc2(output)

print("Output:", output)

tensor([[-0.1013, -1.6739,  0.9861,  ..., -1.5273,  1.5393, -0.5524],
        [ 1.1135, -3.6123,  1.6040,  ..., -4.2860,  4.4062, -0.0993],
        [ 2.3283, -5.5507,  2.2219,  ..., -7.0447,  7.2730,  0.3539],
        [ 3.5431, -7.4890,  2.8398,  ..., -9.8034, 10.1399,  0.8070]],
       grad_fn=<AddmmBackward0>)
Output: tensor([[ 0.1805, -1.3111],
        [-0.4655, -3.1505],
        [-1.1114, -4.9899],
        [-1.7573, -6.8293]], grad_fn=<AddmmBackward0>)
